In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama

In [2]:
llm = Ollama(model="llama2")

In [11]:
%%time
# Not RAG
response = llm.invoke("how can langsmith help with testing?")
print(response["answer"])

Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API key"}\')')
Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API key"}\')')


TypeError: string indices must be integers, not 'str'

In [12]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Shohei_Ohtani")
docs = loader.load()
embeddings = OllamaEmbeddings()

In [ ]:
%%time
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
# HM
# # Set your query here manually
question = "Who is Shohei Otani?"
matched_docs = vector.similarity_search(question, 4)
context = ""
for i, doc in enumerate(matched_docs):
    context = context + doc.page_content + " \n\n "
    print("### ", i)
    print(doc.page_content)

In [5]:
prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

    <context>
    {context}
    </context>

    Question: {input}""")

In [6]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [7]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [8]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API key"}\')')
Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API key"}\')')



Based on the provided context, LangSmith can help with testing in several ways:

1. Debugging: When developing new LLM applications, LangSmith's tracing feature can be enabled by default to quickly understand how the model is performing and debug where it is failing.
2. Test cases: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications. These test cases can be uploaded in bulk, created on the fly, or exported from application traces.
3. Custom evaluations: LangSmith makes it easy to run custom evaluations (both LLM and heuristic-based) to score test results. This helps developers track regressions with respect to their initial test cases.
4. Comparison view: When prototyping different versions of an application, LangSmith's comparison view allows users to see whether or not they have regressed with respect to their initial test cases. This helps diagnose regressions in test scores acro

In [9]:
print(response["answer"])


Based on the provided context, LangSmith can help with testing in several ways:

1. Debugging: When developing new LLM applications, LangSmith's tracing feature can be enabled by default to quickly understand how the model is performing and debug where it is failing.
2. Test cases: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications. These test cases can be uploaded in bulk, created on the fly, or exported from application traces.
3. Custom evaluations: LangSmith makes it easy to run custom evaluations (both LLM and heuristic-based) to score test results. This helps developers track regressions with respect to their initial test cases.
4. Comparison view: When prototyping different versions of an application, LangSmith's comparison view allows users to see whether or not they have regressed with respect to their initial test cases. This helps diagnose regressions in test scores acro